In [ ]:
using DrWatson
@quickactivate "masterarbeit"

In [ ]:
using GLMakie
using LaTeXStrings
using Printf

In [ ]:
Makie.inline!(true)
fontsize_theme = Theme(fontsize=35)
set_theme!(fontsize_theme)

In [ ]:
wblue = Makie.wong_colors()[1]
worange = Makie.wong_colors()[2]
wgreen = Makie.wong_colors()[3]
wpink = Makie.wong_colors()[4]
wlblue = Makie.wong_colors()[5]
worange = Makie.wong_colors()[6]
wyellow = Makie.wong_colors()[7];

In [ ]:
using Revise

In [ ]:
using masterarbeit

In [ ]:
F = Float64

# 2. Pulsed perturbative compton $\omega \rightarrow \omega l$

## 0. Background field

In [ ]:
function abs2plot(f::Function, Δϕ::Real, ls, color)
    res = [abs2(f.(l, Δϕ))/Δϕ for l in ls]
    for i in (1:length(res))
        if isapprox(res[i], 0)
            res[i] = eps()
        end
    end
    lines!(ls, res, label=L"$\Delta\phi = %$Δϕ$", color=color, linewidth=2)
end

function testplot(f::Function, a2=true)
    xs = LinRange(-3.0, 3.0, 10001)
    fig = Figure(size=(1700,1000))
    ax = Axis(fig[1,1], xlabel=L"\ell", ylabel=L"|F(\ell, \Delta\phi)|^2/\Delta\phi", yscale=log10, aspect=1.5)
    ylims!(1E-10, 100)
    
    abs2plot(f, 25.0, xs, wblue)
    abs2plot(f, 50.0, xs, worange)
    abs2plot(f, 250.0, xs, wgreen)
    abs2plot(f, 500.0, xs, wpink)
    fig[1,2] = Legend(fig, ax)
    
    save("generic_spectrum.png", fig)
    fig
end;


testplot(cossquare)

### Plotting functions that are used often

In [ ]:
function draw_heatmap(fig::Figure, keep_axes::Bool, log_colors::Bool, xs, ys, zs, color_extremes, ticknum=5)
    if log_colors
        # dirty hack for log colors https://stackoverflow.com/questions/72803173/logarithmic-scale-for-contourf-colorbar-in-julia-using-makie-jl
        # instead of the real values in zs their log10 is drawn and the colorbar uses the log10 as well, but then the ticks on it are replaced with the corresponding exp10 values
        zsl = [log10(z) for z in zs]
        if keep_axes  # case where the limits for the plot are given
            hm = heatmap!(xs, ys, zsl, colorrange=log10.(color_extremes))
            ticksval = range(log10.(color_extremes)..., length=ticknum)
        else
            hm = heatmap!(xs, ys, zsl, colorrange=(min(zsl[zsl.>log10(eps())]...), max(zsl...)))
            ticksval = range(min(zsl[zsl.>log10(eps())]...), max(zsl...), length=ticknum)
        end
        ticksstring = [@sprintf("%.2E", x) for x in exp10.(collect(ticksval))]  # @sprintf("%.2E", x) converts all numbers to scientific notation (with 2 digits after the decimal point)
        Colorbar(fig[:, end+1], hm, ticks=(ticksval, ticksstring))
        return ticksval, zsl
    else
        if keep_axes
            hm = heatmap!(xs, ys, zs, colorrange = color_extremes)
            ticksval = range(color_extremes..., length=ticknum)
        else
            #hm = heatmap!(xs, ys, zs)
            #ticksval = range(min(zs...), max(zs...), length=ticknum)
            hm = heatmap!(xs, ys, zs, colorrange=(min(zs[zs.>eps()]...), max(zs...)))
            ticksval = range(min(zs[zs.>eps()]...), max(zs...), length=ticknum)
        end
        #ticksstring = [@sprintf("%.2E", x) for x in collect(ticksval)]
        Colorbar(fig[:, end+1], hm)
        return ticksval, zs  # need to return something because this is put into 2 variables in the plotting function
    end
end;

In [ ]:
function draw_l1_line(fig::Figure, keep_axes::Bool, log_colors::Bool, xs, ls, zs, zsl, color_extremes, ticksval, xname, yname, figindex=1, lineres=zs[:, findfirst([isapprox(1.0, l, atol=0.01) for l in ls])])
    # if the plot on the left has a legend then figindex needs to be 4 instead of 3
    
    if log_colors
        ax = Axis(fig[1,figindex], xlabel=xname, ylabel=yname, yscale=log10, yticks=exp10.(collect(ticksval)), ytickformat="{:.2E}")  # "{:.2E}" again converts to scientific notation like in the function above, but the macro wouldn't work in this location
        if keep_axes
            lines!(xs, lineres, color=log10.(lineres), colorrange=log10.(color_extremes))
            ylims!(color_extremes...)
        else
            lines!(xs, lineres, color=log10.(lineres), colorrange=(min(zsl...), max(zsl...)))
            ylims!(eps(), max(zs...))
        end
    else
        ax = Axis(fig[1,figindex], xlabel=xname)
        if keep_axes
            lines!(xs, lineres, color=lineres, colorrange=color_extremes)
            #ylims!(color_extremes...)
        else
            lines!(xs, lineres, color=wblue, label=L"\ell = 1.0")
            #ylims!(min(zs...), max(zs...))
        end
    end
    return ax
end;

## 1. $d\sigma / dldc\theta$

In [ ]:
function lcθplot(ω::Real, Δϕ::Real, steps=1001; lmax=2, video=false, fig::Figure=Figure(size=(750,700)), log_colors=false, return_extremes=false, keep_axes=false, color_extremes=(0.0,1.0))
    if video
        empty!(fig)
        # the title of the plot can't be at the top for videos becase the colorbar will jump into it (for some reason)
        labelpos = 2
    else
        labelpos = 0
    end

    background = (x -> cossquare(x, Δϕ))
    ax = Axis(fig[1,1], xlabel=L"\cos{(\theta)}", ylabel=L"\ell")#, xticks=[-1.0,-0.5,0.0,0.5,1.0])
    cθs = LinRange(-1.0, 1.0, steps)                              # x-axis
    ls = LinRange(lmax/(steps-1), lmax+lmax/(steps-1), steps)     # y-axis
    zs = [dσdldcθ.(cθ, ω, l, background) for cθ in cθs, l in ls]  # z-axis
    #zs = comptonf(vcat(cθs',ls')|>gpu, ω, Δϕ) |> cpu
    #print(zs[1,:])
    if return_extremes
        # for videos where the limits should stay the same over all frames the limits are returned here once
        return (min(zs...), max(zs...))
    end

    ticksval, zsl = draw_heatmap(fig, keep_axes, log_colors, cθs, ls, zs, color_extremes)

    

    # draw a line that corresponds with the values for l=1
    lines!(cθs, (x->1), linestyle=:dash, color=:orange, label="l=1")
    textx = cθs[round(Int, steps/2)]
    texty = ls[findfirst([isapprox(1.0, l, atol=0.01) for l in ls])]
    text!(textx, texty, text=L"$\ell=1.0$", color=worange, align=(:center,:bottom))

    lines!(cθs, (x->0.5), linestyle=:dash, color=worange, label="l=0.5")
    textx = cθs[round(Int, steps/2)]
    texty = ls[findfirst([isapprox(0.5, l, atol=0.01) for l in ls])]
    text!(textx, texty, text=L"$\ell=0.5$", color=worange, align=(:center,:bottom))

    lines!(cθs, (x->1.5), linestyle=:dash, color=worange, label="l=1.5")
    textx = cθs[round(Int, steps/2)]
    texty = ls[findfirst([isapprox(1.5, l, atol=0.01) for l in ls])]
    text!(textx, texty, text=L"$\ell=1.5$", color=worange, align=(:center,:bottom))

    if log_colors
        save("ppcompton_lcosta_log.png", fig)
    else
        save("ppcompton_lcosta.png", fig)
    end
    fig2 = Figure(size=(750,700))

    ax2 = draw_l1_line(fig2, keep_axes, log_colors, cθs, ls, zs, zsl, color_extremes, ticksval, L"\cos{(\theta)}", L"$d\sigma/dldc\theta$")
    lines!(cθs, zs[:, findfirst([isapprox(1.5, l, atol=0.01) for l in ls])], color=worange, label=L"\ell=1.5")
    lines!(cθs, zs[:, findfirst([isapprox(0.5, l, atol=0.01) for l in ls])], color=wgreen, label=L"\ell=0.5")
    lines!(cθs, dσdcθ.(cθs, (ω,)), color=:black, linestyle=:dash, label=L"\mathrm{Klein-Nishina}")
    #colsize!(fig.layout, 3, Aspect(1, 0.6))
    fig2[1,2] = Legend(fig2, ax2)

    if log_colors
        save("ppcompton_lcostb_log.png", fig2)
    else
        save("ppcompton_lcostb.png", fig2)
    end
    fig, fig2
end;

In [ ]:
fig1, fig2 = lcθplot(5.12, 1.43, 1001)
fig1

In [ ]:
fig2

In [ ]:
lcθplot(1.0, 10.0, 1001)

In [ ]:
lcθplot(0.2, 10.0, 1001, log_colors=false)

## 2. $d\sigma / dldt$

In [ ]:
function ltplot(ω::Real, Δϕ::Real, steps=1001; lmin=0.5, lmax=2.0, xmin=-3.5, video=false, fig::Figure=Figure(), log_colors=false, return_extremes=false, keep_axes=false, color_extremes=(0.0,1.0))
    if video
        empty!(fig)
        labelpos = 2
    else
        labelpos = 0
    end

    background = (x -> cossquare(x, Δϕ))
    Axis(fig[1,1], xlabel=L"t", ylabel=L"l", title=L"$d\sigma/dldt$")
    xlims!(low = xmin)
    cθs = LinRange(-1.0, 1.0, steps)
    ts = [2*ω*calc_ω2(ω,cθ)*(cθ-1) for cθ in cθs]
    ls = LinRange(lmin, lmax, steps)
    zs = [dσdldt.(cθ, ω, l, background) for cθ in cθs, l in ls]

    if return_extremes
        return (min(zs...), max(zs...))
    end

    ticksval, zsl = draw_heatmap(fig, keep_axes, log_colors, ts, ls, zs, color_extremes)

    lines!(ts, (x->1), linestyle=:dash, color=:orange, label="l=1")
    textx = (ts[1]-ts[end])/2
    texty = ls[findfirst([isapprox(1.0, l, atol=0.01) for l in ls])]
    text!(textx, texty, text=L"$l=1$", color=:orange, align=(:center,:bottom))

    draw_l1_line(fig, keep_axes, log_colors, cθs, ls, zs, zsl, color_extremes, ticksval, L"t", L"$d\sigma/dldt$")

    ωround = rpad(round(ω, digits=4), 6, "0")
    Δϕround = rpad(round(Δϕ, digits=4), 6, "0")
    captiontext = latexstring(L"ppCompton diff. cross section for $\omega = $", ωround, L", $\Delta\phi = $", Δϕround)
    Label(fig[labelpos,:], text=captiontext, fontsize=22)

    save("compton_lt.png", fig)
    fig
end;

In [ ]:
ltplot(0.1, 10.0)

In [ ]:
ltplot(1.0, 10.0)

## 3. $d\sigma / d\omega'dc\theta$ 

In [ ]:
function draw_l_line(number::Real, cθs, ω, steps, maxy, color, label=L"\ell = %$number")
    values = [calc_ω2(ω*number, cθ) for cθ in cθs]
    for i in 1:length(values)
        if values[i] > maxy
            values[i] = maxy
        end
        if values[i] < 0
            values[i] = maxy
        end
    end
    lines!(cθs, values, linestyle=:dash, color=color, label=label)
end

#=function draw_os_ex(ω, maxy, cθs) #needs l
    res = [t_ex(ω) / (2*ω*(cos-1)) for cos in cθs]
    for i in 1:length(res)
        if res[i] > maxy
            res[i] = maxy
        end
    end
    lines!(cθs, res, linestyle=:dash, color=:black, label=L"\omega'_{ex}")
end=#

function ω2cθplot(ω::Real, Δϕ::Real, steps=1001; video=false, fig::Figure=Figure(size=(1200,1000)), log_colors=false, unsafe=false, return_extremes=false, return_ω2max=false, keep_axes=false, color_extremes=(0.0,1.0), limit_ω2=false, ω2max=1.0)
    if video
        empty!(fig)
        labelpos = 2
    else
        labelpos = 0
    end

    background = (x -> cossquare(x, Δϕ))
    ax = Axis(fig[1,1], xlabel=L"\cos{(\theta)}", ylabel=L"\omega'", aspect=1)
    cθs = LinRange(-1, 0.998, steps)
    l_line = [calc_ω2(ω, cθ) for cθ in cθs] ###not independant??
    ω2s = LinRange(l_line[1], l_line[end], steps) #why?
    zs = [dσdω2dcθ.(cθ, os, ω, background) for cθ in cθs, os in ω2s]
    maxy = ω2s[end]
    if return_ω2max
        return maxy
    end
    
    # calculate the max allowed ω' for each cθ
    ωex = [m_e/(1-cθ) for cθ in cθs]
    for i in 1:length(ωex)
        if ωex[i] > maxy
            ωex[i] = maxy
        end
    end
    
    if !unsafe
        for i in 1:length(cθs)
            for j in 1:length(ω2s)
                if ω2s[j] > ωex[i]
                    zs[i,j] = eps()
                end
            end
        end
    end

    if return_extremes
        return (min(zs[zs.>eps()]...), max(zs...))  # need to exclude the eps because that as minimum value would make the log plot in videos look bad
    end

    ticksval, zsl = draw_heatmap(fig, keep_axes, log_colors, cθs, ω2s, zs, color_extremes)

    draw_l_line(1, cθs, ω, steps, maxy, :orange)
    draw_l_line(-1, cθs, ω, steps, maxy, :green)
    draw_l_line(5000000, cθs, ω, steps, maxy, :white, L"\ell \rightarrow \infty")
    #draw_l_line(0.3, cθs, ω, steps, maxy, :pink)
    #draw_os_ex(ω, maxy, cθs)
    #lines!(cθs, ωex, color=:white)  # max allowed ω' line

    if limit_ω2
        ylims!(low=0.0, high=ω2max)
    end

    fig[1,3] = Legend(fig, ax)

    #draw_l1_line(fig, keep_axes, log_colors, cθs, ω2s, zs, zsl, color_extremes, ticksval, L"\cos{(\theta)}", L"$d\sigma/d\omega'd\cos{(\theta)}$", 4, dσdω2dcθ.(cθs, l_line, ω, background))

    ωround = rpad(round(ω, digits=4), 6, "0")
    Δϕround = rpad(round(Δϕ, digits=4), 6, "0")
    #captiontext = latexstring(L"ppCompton diff. cross section for $\omega = $", ωround, L", $\Delta\phi = $", Δϕround)
    #Label(fig[labelpos,:], text=captiontext, fontsize=22)

    if log_colors
        save("ppcompton_w2cost_log.png", fig)
    else
        save("ppcompton_w2cost.png", fig)
    end
    fig
end;

In [ ]:
ω2cθplot(5.12, 1.43, log_colors=false)

In [ ]:
#ω2cθplot(0.1, 10.0, log_colors=false)

In [ ]:
ω2cθplot(5.12, 10.0, log_colors=true)

In [ ]:
ω2cθplot(2.0, 1.43, log_colors=true, limit_ω2=true, ω2max=2.5)

In [ ]:
ω2cθplot(2.0, 1.43, log_colors=false)

## 4. $d\sigma / d\chi dc\theta$

In [ ]:
function draw_l_line_vex(number::Real, cθs, ω, steps, maxy, color, label=L"l = %$number")
    values = [calc_ω2(ω*number, cθ)/calc_ω2ex(cθ) for cθ in cθs]
    for i in 1:length(values)
        if values[i] > maxy
            values[i] = maxy
        end
        if values[i] < 0
            values[i] = maxy
        end
    end
    lines!(cθs, values, linestyle=:dash, color=color, label=label)
end

function ω2_ex_cθplot(ω::Real, Δϕ::Real, steps=1001; video=false, fig=Figure(size=(1200,1000)), log_colors=false, return_extremes=false, keep_axes=false, color_extremes=(0.0,1.0))
    if video
        empty!(fig)
        labelpos = 2
    else
        labelpos = 0
    end

    ymax = 1
    background = (x -> cossquare(x, Δϕ))
    ax = Axis(fig[1,1], xlabel=L"\cos{(\theta)}", ylabel=L"\overline{\omega'}", yticks=[0.0,0.2,0.4,0.6,0.8,1.0], aspect=1)
    cθs = LinRange(-1, 0.998, steps)
    ωex = calc_ω2ex(collect(cθs))
    #y_axis = LinRange(ymax/(steps-1), ymax+ymax/(steps-1), steps) 
    y_axis = LinRange(ymax/(steps-1)*3, ymax-ymax/(steps-1), steps) # need to exclude 0 because that is under a fraction in dσdω2dcθ_ex and exclude ωex (y=1) because the values are negative there
    l_line = [calc_ω2(ω, cθ) for cθ in cθs]./ωex
    ω2s_scaled = [y*ωexi for y in y_axis, ωexi in ωex] #why
    zs = [dσdω2dcθ_ex([cθ], [ω2], ω, background)[1] for cθ in cθs, ω2 in y_axis]
    if return_extremes
        return (min(zs...), max(zs...))
    end
    maxy = ω2s_scaled[end]

    ticksval, zsl = draw_heatmap(fig, keep_axes, log_colors, cθs, y_axis, zs, color_extremes)


    ωround = rpad(round(ω, digits=4), 6, "0")
    Δϕround = rpad(round(Δϕ, digits=4), 6, "0")
    captiontext = latexstring(L"ppCompton diff. cross section for $\omega = $", ωround, L", $\Delta\phi = $", Δϕround)
    #Label(fig[labelpos,:], text=captiontext, fontsize=22)
    
    if log_colors
        save("ppcompton_w2bar_log.png", fig)
    else
        save("ppcompton_w2bar.png", fig)
    end
    save("ppcompton_w2bar.png", fig)
    fig
end;

$\omega'/\omega'_{ex} := \overline{\omega'}$

In [ ]:
ω2_ex_cθplot(0.1, 10.0)

In [ ]:
ω2_ex_cθplot(5.12, 10.0)

In [ ]:
ω2_ex_cθplot(0.1, 10.0, log_colors=true)

In [ ]:
ω2_ex_cθplot(5.12, 10.0, log_colors=true)